In [10]:
import leafmap.foliumap as leafmap
import geopandas as gpd

# Standard library imports
import json
import os
from pathlib import Path
import re
import urllib3


# Third-party imports
import dask
import dask.distributed
import dask.utils
from datacube.utils.cog import write_cog
from dotenv import load_dotenv
import geopandas as gpd
import numpy as np
from odc.stac import configure_rio, stac_load
import pandas as pd
import planetary_computer as pc
from pystac_client import Client
import rasterio as rio
from rasterio.mask import mask as rio_mask
from rasterio.session import AWSSession
import xarray as xr
from IPython.display import display



print("Load environment variables from .env file.")
load_dotenv()
USGS_API_KEY = os.environ["USGS_API_KEY"]
USGS_TOKEN_NAME = os.environ["USGS_TOKEN_NAME"]
USGS_USERNAME = os.environ["USGS_USERNAME"]
USGS_PASSWORD = os.environ["USGS_PASSWORD"]
AWS_ACCESS_KEY = os.environ["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = os.environ["AWS_SECRET_KEY"]
NASA_EARTHDATA_S3_ACCESS_KEY = os.environ["NASA_EARTHDATA_S3_ACCESS_KEY"]
NASA_EARTHDATA_S3_SECRET_KEY = os.environ["NASA_EARTHDATA_S3_SECRET_KEY"]
NASA_EARTHDATA_S3_SESSION = os.environ["NASA_EARTHDATA_S3_SESSION"]
NASA_EARTHDATA_USERNAME = os.environ["NASA_EARTHDATA_USERNAME"]
NASA_EARTHDATA_PASSWORD = os.environ["NASA_EARTHDATA_PASSWORD"]

DATA_DIR = Path(r"C:\Users\Peter\gh\rasmussen-705.603\data\FinalProject")
RES = 10
STAC_ENDPOINT = "https://planetarycomputer.microsoft.com/api/stac/v1"
COLLECTIONS = ["sentinel-2-l2a"]
COLLECTION_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "qa"]
OUTPUT_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "ndvi", "qa"]

os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "FALSE"


# Define directory paths
raw_dir = DATA_DIR / "raw"
interim_dir = DATA_DIR / "interim"
processed_dir = DATA_DIR / "processed"
region_dir = raw_dir / "regions"
maxar_cogs_dir = interim_dir / "maxar_cogs"
maxar_cogs_dir.mkdir(exist_ok=True, parents=True)
dst_dir = interim_dir / "cogs"
dst_dir.mkdir(exist_ok=True, parents=True)

Load environment variables from .env file.


In [2]:
leafmap.maxar_collections()

['Gambia-flooding-8-11-2022',
 'Hurricane-Fiona-9-19-2022',
 'Hurricane-Ian-9-26-2022',
 'Indonesia-Earthquake22',
 'Kahramanmaras-turkey-earthquake-23',
 'New-Zealand-Flooding22',
 'New-Zealand-Flooding23',
 'Sudan-flooding-8-22-2022',
 'afghanistan-earthquake22',
 'cyclone-emnati22',
 'kentucky-flooding-7-29-2022',
 'pakistan-flooding22',
 'southafrica-flooding22',
 'tonga-volcano21',
 'volcano-indonesia21',
 'yellowstone-flooding22']

In [26]:
regions

,event_key,a2,event_date,s2_start,s2_end,geometry
maxar_event,,,,,,
afghanistan-earthquake22,af-kharkamar-2022,af,2022-06-21,2016-01-01,2023-04-29,"POLYGON ((69.81754 32.97436, 69.81754 32.96162..."
Gambia-flooding-8-11-2022,gm-kanifing-2022,gm,2022-08-09,2016-01-01,2023-04-29,"POLYGON ((-16.68547 13.47630, -16.68547 13.423..."
Indonesia-Earthquake22,in-cianjur-2022,in,2022-11-21,2016-01-01,2023-04-29,"POLYGON ((107.11600 -6.79750, 107.11600 -6.849..."
cyclone-emnati22,mg-farafangana-2022,mg,2022-02-22,2016-01-01,2023-04-29,"POLYGON ((47.83751 -22.82598, 47.83751 -22.810..."
Kahramanmaras-turkey-earthquake-23,tr-islahiye-2023,tr,2023-02-06,2016-01-01,2023-04-29,"POLYGON ((36.64975 37.03872, 36.61874 37.03872..."
NaN,us-baltimore-9999,us,2021-02-06,2016-01-01,2023-04-29,"POLYGON ((-76.65257 39.31924, -76.65257 39.270..."


In [28]:
import leafmap
regions = gpd.read_file(raw_dir / "regions.geojson").dropna().set_index("maxar_event")
events = regions.index.values


for event in events:
    collections = leafmap.maxar_child_collections(event)
    print(f"[Download Maxar][{event}]: Number of collections: {len(collections)}")
    geoframes = []
    
    for collection in collections:
        geoframe = leafmap.maxar_items(
            collection_id=event, 
            child_id=collection,
            return_gdf=True,
        )
        geoframes.append(geoframe)
    gdf = pd.concat(geoframes)
    intersecting_tiles = gdf.sjoin(regions, how="inner", op="intersects")
    
    print(f"[Download Maxar][{event}]: Download TIFs.")
    for tif_url in intersecting_tiles["visual"].values:
        # Build file path
        scene_date = re.search(r"\d{4}-\d{2}-\d{2}", tif_url).group(0)
        scene_id = Path(tif_url).stem
        fn = f"{event}_{scene_date}_{scene_id}.tif"
        dst_path = dst_dir / fn
        
        # Download TIF
        http = urllib3.PoolManager()
        response = http.request('GET', tif_url)

        print(f"[Download Maxar][{event}]: Save {dst_path.name}")
        # Write TIF
        with open(dst_path, 'wb') as f:
            f.write(response.data)    
    

afghanistan-earthquake22
The number of collections: 14
	10400100737B8400
	105001002998D400
	10300100C084CC00
	10300100C04CC000
	10300100BFCFAF00
	10300100C028B900
	10300100C0C06900
	1040010077468F00
	10400100777CAA00
	10300100D4928800
	10300100D5B4AB00
	1040010077B01400
	10400100779DDF00
	105001002CB61600
	Save afghanistan-earthquake22_2022-02-04_10400100737B8400-visual.tif.
	Save afghanistan-earthquake22_2022-01-31_105001002998D400-visual.tif.
Gambia-flooding-8-11-2022
The number of collections: 6
	105001002BD68F00
	10300100CFC9A500
	1040010073D77D00
	104001007B1A7600
	104001007C806A00
	104001007A565700
	Save Gambia-flooding-8-11-2022_2022-05-19_105001002BD68F00-visual.tif.
	Save Gambia-flooding-8-11-2022_2022-05-19_105001002BD68F00-visual.tif.
	Save Gambia-flooding-8-11-2022_2022-05-19_105001002BD68F00-visual.tif.
	Save Gambia-flooding-8-11-2022_2022-03-15_10300100CFC9A500-visual.tif.
	Save Gambia-flooding-8-11-2022_2022-03-15_10300100CFC9A500-visual.tif.
	Save Gambia-flooding-8-11-2

'105001002998D400-visual.tif'

afghanistan-earthquake22_2022-02-04_10400100737B8400-visual.tif
afghanistan-earthquake22_2022-01-31_105001002998D400-visual.tif


In [23]:
import boto3

s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
url = 'https://maxar-opendata.s3.amazonaws.com/events/afghanistan-earthquake22/ard/42/120200201331/2022-01-31/105001002998D400-visual.tif'

bucket = url.split('/')[2]
key = '/'.join(url.split('/')[3:])

response = s3.get_object(Bucket=bucket, Key=key)

with open('105001002998D400-visual.tif', 'wb') as f:
    f.write(response['Body'].read())

NoSuchBucket: An error occurred (NoSuchBucket) when calling the GetObject operation: The specified bucket does not exist

In [ ]:
# 8 and 12 are the best
m = leafmap.Map()
for geoframe in geoframes:
    m.add_gdf(geoframe, layer_name=geoframe.catalog_id[0])
m.add_gdf(regions.loc[[event]], layer_name=event)
m

In [4]:
import datacube
# from stac import STAC
import requests

# Define the area of interest (AOI) as a GeoJSON Polygon
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [
                68.77120971679688,
                37.065393740039164
            ],
            [
                69.50531005859375,
                37.065393740039164
            ],
            [
                69.50531005859375,
                37.54569422279954
            ],
            [
                68.77120971679688,
                37.54569422279954
            ],
            [
                68.77120971679688,
                37.065393740039164
            ]
        ]
    ]
}

# Connect to the Data Cube
dc = datacube.Datacube(app='afg-earthquake-2022')

# Define the product and query the Data Cube for available datasets
product = 'maxar-worldview3'
datasets = dc.find_datasets(product=product, geopolygon=aoi)

# Print the number of available datasets
print(f'Found {len(datasets)} datasets')

# # Define the time range of interest
# start_date = '2022-05-01'
# end_date = '2022-06-01'

# # Filter the datasets by the time range of interest
# datasets = list(filter(lambda ds: ds.time.begin < end_date and ds.time.end > start_date, datasets))

# # Print the number of datasets that match the time range
# print(f'Found {len(datasets)} datasets between {start_date} and {end_date}')

# # Get the STAC URL for each dataset
# stac_urls = [ds.metadata_doc['stac_extensions']['eo']['view:single']['assets']['analytic']['href'] for ds in datasets]

# # Download the GeoTIFF files for each dataset
# for url in stac_urls:
#     r = requests.get(url, allow_redirects=True)
#     with open(f'{product}_{start_date}_{end_date}.tif', 'wb') as f:
#         f.write(r.content)

ValueError: No ODC environment, checked configurations for ['default', 'datacube']

In [5]:
import dask
print("Instantiate dask client.")
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)
display(client)

Instantiate dask client.


AttributeError: module 'dask' has no attribute 'distributed'

In [29]:
# create a client instance for the Maxar STAC API
client = Client.open("https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json")#Client.open("https://api.maxar.com/catalog/v3")

# search for the 'afghanistan-earthquake22' dataset
search = client.search(
    # collections=["maxar-worldview"],
    # bbox=[69.0, 32.0, 75.0, 37.0],
    # datetime="2022-05-01T00:00:00Z/2022-05-31T23:59:59Z",
    # query={"eo:cloud_cover": {"lt": 5}},
    limit=10,
    sortby=["-properties.datetime"],
)

# print the number of items found
# print(f"Found {len(items)} items")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
from pystac_client import Client
url = 'https://maxar-opendata.s3.amazonaws.com/events/catalog.json'
client = Client.open(url)
client.search(datetime='2023-02-01/2023-02-28')

NotImplementedError: This catalog does not support search because it does not conform to "ConformanceClasses.ITEM_SEARCH"

In [55]:
# madagascar collection = collections[2] == '1040010068610000'
# afghanistan collection = collections[12] =='10400100779DDF00'

The number of collections: 14


,geometry,datetime,platform,gsd,ard_metadata_version,catalog_id,utm_zone,quadkey,view:off_nadir,view:azimuth,...,view:sun_azimuth,view:sun_elevation,proj:epsg,proj:geometry,grid:code,proj:bbox,tile:data_area,tile:clouds_area,tile:clouds_percent,visual
0,"POLYGON ((69.75352 33.29793, 69.75318 33.25531...",2022-06-24T06:35:58Z,WV03,0.55,0.0.1,10400100779DDF00,42,120200023323,42.8,256.6,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[570154.4...",MXRA-Z42-120200023323,"566974.7924804688,3679843.75,570156.25,3684569...",13.8,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
1,"POLYGON ((69.80730 33.30395, 69.80730 33.30395...",2022-06-24T06:35:58Z,WV03,0.55,0.0.1,10400100779DDF00,42,120200023330,43.0,256.6,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[575155.6...",MXRA-Z42-120200023330,"572101.7456054688,3684843.75,575156.25,3685274...",0.6,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
2,"POLYGON ((69.86105 33.30994, 69.86097 33.29971...",2022-06-24T06:35:58Z,WV03,0.56,0.0.1,10400100779DDF00,42,120200023331,43.2,256.7,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[580154.1...",MXRA-Z42-120200023331,"574843.75,3684843.75,580156.25,3685978.6987304688",4.0,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
3,"POLYGON ((69.79822 33.30294, 69.80729 33.30288...",2022-06-24T06:35:58Z,WV03,0.55,0.0.1,10400100779DDF00,42,120200023332,43.0,256.6,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[574311.8...",MXRA-Z42-120200023332,"569843.75,3679843.75,575156.25,3685156.25",26.8,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...
4,"POLYGON ((69.80394 33.30291, 69.80350 33.25499...",2022-06-24T06:35:58Z,WV03,0.56,0.0.1,10400100779DDF00,42,120200023333,43.2,256.7,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[574843.7...",MXRA-Z42-120200023333,"574843.75,3679843.75,580156.25,3685156.25",28.2,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...


NameError: name 'raw_dir' is not defined

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [34]:
gdf.visual.iloc[0]

'https://maxar-opendata.s3.amazonaws.com/events/afghanistan-earthquake22/ard/42/120200023323/2022-06-24/10400100779DDF00-visual.tif'

In [13]:
gdf.to_crs(4326).to_file("em.geojson", driver="GeoJSON")

In [19]:
gdf.head(1)

,geometry,datetime,platform,gsd,ard_metadata_version,catalog_id,utm_zone,quadkey,view:off_nadir,view:azimuth,...,view:sun_azimuth,view:sun_elevation,proj:epsg,proj:geometry,grid:code,proj:bbox,tile:data_area,tile:clouds_area,tile:clouds_percent,visual
0,"POLYGON ((69.75352 33.29793, 69.75318 33.25531...",2022-06-24T06:35:58Z,WV03,0.55,0.0.1,10400100779DDF00,42,120200023323,42.8,256.6,...,130.2,75.9,32642,"{'type': 'Polygon', 'coordinates': [[[570154.4...",MXRA-Z42-120200023323,"566974.7924804688,3679843.75,570156.25,3684569...",13.8,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...


In [10]:
images = gdf['visual'].tolist()
images[:5]

['https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010322/2023-02-07/1050050044DE7E00-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010323/2023-02-07/1050050044DE7E00-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010332/2023-02-07/1050050044DE7E00-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010333/2023-02-07/1050050044DE7E00-visual.tif',
 'https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133011222/2023-02-07/1050050044DE7E00-visual.tif']

In [18]:
m = leafmap.Map()
m.add_cog_layer('https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010322/2023-02-07/1050050044DE7E00-visual.tif')
m

In [17]:
url = 'https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133010322/2023-02-07/1050050044DE7E00.json'
m2 = leafmap.Map()
m2.add_stac_layer(url, name="False color")
m2

In [11]:
url = "https://maxar-opendata.s3.amazonaws.com/events/catalog.json"

['1050050044DE7E00',
 '1050050044DE7F00',
 '1040010082B85E00',
 '1040010082698700',
 '10300500D8F90C00',
 '104001007E564300',
 '104001007DAE2D00',
 '10500500F0DD9700',
 '10300100DD62DB00',
 '10300100E1700300',
 '10300100E18A1000',
 '1040010077137900',
 '1040010067C49900',
 '10300100DF1DB000',
 '10300100D6740900',
 '10300500BFF94800',
 '10200100B5C3A800',
 '104001008058CD00',
 '10300100DF6BFE00',
 '10300100D76F1300',
 '10300100D6789800',
 '10300500BFF95000',
 '10300100DC08C900',
 '10300100DB913300',
 '104001006D138700',
 '10300100D797E100',
 '10300100DF069700',
 '10300500BFF94C00',
 '10300100DFA9DD00',
 '10300100E0287700',
 '10300500D9F8CC00',
 '10300500D9F8D100',
 '10300500D9F8D500',
 '1040010082B85E00',
 '10300500D9F8D200',
 '10300500D9F8D400',
 '10300100E2B30500',
 '10300500D9F8E600',
 '10300100E2901100',
 '10300100E131D000',
 '10300100E29F9100',
 '10300100E131D000',
 '10300100E18CB600',
 '10300100E1B9D900',
 '10300100E2A73B00',
 '104001008314FC00',
 '10300100E2226200',
 '10300100E29

In [20]:
import pystac
import requests

# Define the STAC API URL for the Maxar Open Data Catalog
catalog_url = "https://maxar-opendata.s3.amazonaws.com/events/catalog.json"

# Load the catalog into a pystac Catalog object
catalog = pystac.Catalog.from_file(catalog_url)

# Get the specific event you want to download data for
event_id = "cyclone-emnati22"
event = catalog.get_item(event_id)

# Get the assets for the event
assets = event.assets

# Select the asset you want to download (e.g., the RGB image)
asset_key = "image:visual"
asset = assets[asset_key]

# Download the asset
response = requests.get(asset.href)

# Save the asset to disk
with open(f"{event_id}.tif", "wb") as f:
    f.write(response.content)

AttributeError: 'NoneType' object has no attribute 'assets'

In [127]:
import pystac
import requests

# Define the STAC API URL for the Maxar Open Data Catalog
catalog_url = "https://prd-tnm.s3.amazonaws.com/maxar-catalog/catalog.json"

# Load the catalog into a pystac Catalog object
catalog = pystac.Catalog.from_file(catalog_url)

# Get the specific event you want to download data for
event_id = "2021-biden-inauguration"
event = catalog.get_item(event_id)

# Get the assets for the event
assets = event.assets

# Select the asset you want to download (e.g., the RGB image)
asset_key = "image:visual"
asset = assets[asset_key]

# Download the asset
response = requests.get(asset.href)

# Save the asset to disk
with open(f"{event_id}.tif", "wb") as f:
    f.write(response.content)

Exception: Could not read uri https://prd-tnm.s3.amazonaws.com/maxar-catalog/catalog.json

# Start

In [137]:
from datetime import datetime
import json
import os
from pathlib import Path

import boto3
import dask

from dotenv import load_dotenv
import geopandas as gpd
import getpass
import hvplot.xarray
from joblib import Parallel, delayed
import pandas as pd
import planetary_computer
from pystac_client import Client
import rasterio as rio
from rasterio.mask import mask as rio_mask
from rasterio.session import AWSSession
import rioxarray
import stackstac
import xarray as xr

def get_raster(s3_link, geom, chunks="auto", drop=True):
    try:
        da = rioxarray.open_rasterio(s3_link, chunks="auto").squeeze('band', drop=True).rio.clip([geom])
        print("*", end="")
        return da
    except Exception as e:
        print(f"\nLink failed: {s3_link}")
        print(e)
        return None


def time_index_from_filenames(file_links):
    '''
    Helper function to create a pandas DatetimeIndex
    '''
    return [datetime.strptime(f.split('.')[-5], '%Y%jT%H%M%S') for f in file_links]


load_dotenv()
USGS_API_KEY = os.environ["USGS_API_KEY"]
USGS_TOKEN_NAME = os.environ["USGS_TOKEN_NAME"]
USGS_USERNAME = os.environ["USGS_USERNAME"]
USGS_PASSWORD = os.environ["USGS_PASSWORD"]
AWS_ACCESS_KEY = os.environ["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = os.environ["AWS_SECRET_KEY"]
NASA_EARTHDATA_S3_ACCESS_KEY = os.environ["NASA_EARTHDATA_S3_ACCESS_KEY"]
NASA_EARTHDATA_S3_SECRET_KEY = os.environ["NASA_EARTHDATA_S3_SECRET_KEY"]
NASA_EARTHDATA_S3_SESSION = os.environ["NASA_EARTHDATA_S3_SESSION"]
NASA_EARTHDATA_USERNAME = os.environ["NASA_EARTHDATA_USERNAME"]
NASA_EARTHDATA_PASSWORD = os.environ["NASA_EARTHDATA_PASSWORD"]

DATA_DIR = Path(r"C:\Users\Peter\gh\rasmussen-705.603\data\ResearchPaper")
AOI = "baltimore_cnn"

data_dir = DATA_DIR / AOI

raw_dir = data_dir / "raw"
interim_dir = data_dir / "interim"

raw_dir.mkdir(exist_ok=True, parents=True)
interim_dir.mkdir(exist_ok=True, parents=True)

ModuleNotFoundError: No module named 'rioxarray'

In [130]:
import pystac
from pystac_client import Client

# Set up the client
client = Client.open('https://api.stac.maxar.com/v2')

# # Get the catalog for Maxar open data
# catalog = client.get_catalog()

# # Get the collection for Maxar open data's WorldView imagery
# collection = catalog.get_child('maxar-worldview')

# # Get the items for the "cyclone-emnati22" event
# items = collection.get_items(filter={
#     'collections': 'maxar-worldview',
#     'properties': {
#         'event': 'cyclone-emnati22'
#     }
# })

# # Print the number of items found
# print(f"Found {len(items)} items for 'cyclone-emnati22':")

# # Print the IDs of the items found
# for item in items:
#     print(f"- {item.id}")

APIError: HTTPSConnectionPool(host='api.stac.maxar.com', port=443): Max retries exceeded with url: /v2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001EE8EBDE6B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))